# 비트맵
-  비트맵 인덱싱은 카디널리티가 낮은 스칼라 필드에서 쿼리 성능을 개선하기 위해 고안된 효율적인 인덱싱 기법
- 카디널리티는 필드에 있는 교유 값의 수를 나타냄
- 고유 요소가 적은 필드는 카디널리티가 낮은 필드로 간주
- 필드 값을 간결한 이진 형식으로 표현하고 효율적인 비트 단위 연산을 수행해 스칼라 쿼리의 검색 시간을 단축하는 데 도움이 됨
- 다른 유형의 인덱스에 비해 비트맵 인덱스는 일반적으로 카디널리티가 낮은 필드를 처리할 때 공간 효율성이 높고 쿼리 속도가 빠름

참교자료: https://milvus.io/docs/ko/v2.5.x/bitmap.md

## 비트맵 인덱스 만들기
- Milvus에서 비트맵 인덱스를 생성하려면 create_index() 방법을 사용하고 index_type 파라미터를 "BITMAP"로 설정

In [7]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"  # 또는 실제 Milvus 서버 주소
)

index_params = client.prepare_index_params()

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=False,
)

schema.add_field(field_name="index", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="category", datatype=DataType.INT64)
schema.add_field(field_name="scalar_2", datatype=DataType.INT32)

client.create_collection(
    collection_name="my_collection",
    schema=schema
)

index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="category",
    index_type="BITMAP",
    index_name="category_bitmap_index"
)

client.create_index(
    collection_name="my_collection",
    index_params=index_params
)

## 제한 사항
- 비트맵 인덱스는 기본 키가 아닌 스칼라 필드에 대해서만 지원
- 필드의 데이터 유형은 다음 중 하나여야 함
    - BOOL, INT8, INT16, INT32, INT64, VARCHAR
    - ARRAY(요소는 BOOL, INT8, INT16, INT32, INT64, VARCHAR) 중 하나여야 함
- 비트맵 인덱스는 다음 데이터 유형을 지원하지 않음
    - FLOAT, DOUBLE: 부동소수점 유형은 비트맵 인덱스의 이진 특성과 호환되지 않음
    - JSON: JSON 데이터 유형은 비트맵 인덱스를 사용하여 효율적으로 표현할 수 없는 구조를 가지고 있음
- 비트맵 인덱스는 카디널리티가 높은 필드(고유값이 많은)에는 적합하지 않음
    - 500 미만일 때가 효과적
    - 카디널리티가 이 임계값 이상으로 증가하면 비트맵 인덱스의 성능 이점이 줄어들고 스토리지 오버헤드가 크게 증가
    - 카디널리티가 높은 필드의 경우, 특정 사용 사례와 쿼리 요구 사항에 따라 역 인덱스와 같은 대체 인덱싱 기술을 사용하는 것을 고려